In [11]:
import pandas as pd
import numpy as np

In [12]:
point_panel_in = "../data/final/point_panel.parquet"
tract_panel_in = "../data/final/tract_panel.parquet"

panel_out = "../data/final/model_data.parquet"

# Pipeline In

In [13]:
point_panel = pd.read_parquet(point_panel_in)
tract_panel = pd.read_parquet(tract_panel_in)

In [14]:
tract_panel = tract_panel.loc[~tract_panel.is_weekend]
point_panel = point_panel.loc[~point_panel.is_weekend]

In [15]:
def get_model_data(panel, transit):
    model_data = panel.query(f'transit == "{transit}"')
    empty_units = model_data.groupby('id')['rides'].transform('sum') == 0
    model_data = model_data[~empty_units]
    return model_data

In [16]:
model_data = pd.concat([
    get_model_data(tract_panel, "uber").assign(id = lambda x: x['id'].astype(str)),
    get_model_data(point_panel, "train").assign(id = lambda x: x['id'].astype(str)),
    get_model_data(point_panel, "bike").assign(id = lambda x: x['id'].astype(str)),
])

In [17]:
# Add log distance.
for col in model_data.columns:
    if 'distance' in col:
        model_data['log_' + col] = model_data[col].apply(np.log1p)

# Pipeline Out

In [18]:
model_data.to_parquet(panel_out, index=False)